In [1]:
import os
import math
import numpy as np
import pandas as pd
import networkx as nx

In [2]:
# Load the data

path = 'C:\\Users\\kongl\\Desktop\\NYU Shanghai\\Fall 2021\\Network Analytics\\Project\data.csv'
df = pd.read_csv(path)

## Data Check

First, we want to check if the experiment is correctly operated according to the game design.<br>
If nothing went wrong, we expect to see a ***robot_distance*** list of [0,1,2,4,5] for each participant. 

In [3]:
# Build a function that check if all participants were tested according to experiment design

def check_exp(df):
    
    '''
    The input of the function is the experiment dataset.
    The output of the function are data of participants whose treatment deviated from the experiment design. 
    '''
    
    abnormal_participants = []
    
    for i in range(int(len(df)/5)):
       start = 5*i
       end = 5*(i+1)
       test = list(df["robot_distance"][start:end]) == [0,1,2,4,5]
       if test == False:
           print(test, start, end)
           df_participant = df.iloc[start:end,:]
           abnormal_participants.append(df_participant)
    
    return abnormal_participants

In [4]:
df_abnormal = check_exp(df)
print(df_abnormal[0][["token", "robot_distance"]])
print(df_abnormal[1][["token", "robot_distance"]])

False 125 130
False 26325 26330
                    token  robot_distance
125  acsrrvomanbkdmaseuru               0
126  acsrrvomanbkdmaseuru               0
127  acsrrvomanbkdmaseuru               1
128  acsrrvomanbkdmaseuru               1
129  acsrrvomanbkdmaseuru               2
                      token  robot_distance
26325  uddgqjkxurpcrmsfngsj               0
26326  uddgqjkxurpcrmsfngsj               0
26327  uddgqjkxurpcrmsfngsj               1
26328  uddgqjkxurpcrmsfngsj               2
26329  uddgqjkxurpcrmsfngsj               5


We can see that, for two participants, the robot profiles were not generated correctly.<br>
We might want to drop their data.<br>
We don't drop them for now.

## Data Modification

In [5]:
# Get all participant data we are interested in

df_participant = df[df['robot_distance'] == 0]
df_participant = df_participant.drop([126, 26326]) # drop two duplicated rows
select_var = ["token", "age", "gender", "marital_status", "state", "region", "answer.Risk", "world"]
df_participant = df_participant[select_var]
df_participant.reset_index(inplace = True)

In [6]:
# Get a list of all tokens representing participants

list_participants = list(df_participant['token'])

## Data Engineering

Next, we want to calculate information required to build a network graph.

In [7]:
import itertools # for pairing participants

In [8]:
# Get all possible participant pairs 

paired_participants = list(itertools.combinations(list_participants,2))
df_pairs = pd.DataFrame(paired_participants)

In [9]:
# Restructure the dataframe for edge storage

df_edges = pd.DataFrame(df_pairs.groupby([0,1]).size())
df_edges.rename(columns={0:'social_dist'}, inplace=True)
df_edges.reset_index(inplace=True)
df_edges

,0,1,social_dist
0,aabfstzqyjjpmtxbmvkg,aaevujyayfdsvfsdtdju,1
1,aabfstzqyjjpmtxbmvkg,aafsazabzmeyvljckcoi,1
2,aabfstzqyjjpmtxbmvkg,aaloodukoptypzscbnrn,1
3,aabfstzqyjjpmtxbmvkg,aappcdtbnehthfwdzjjh,1
4,aabfstzqyjjpmtxbmvkg,aaqyqficqrloahhcvyay,1
...,...,...,...
22535536,zzloxgvjuwcsnrcncyhe,zzucufmtbxmihtplavko,1
22535537,zzloxgvjuwcsnrcncyhe,zzzfvqpmrvtwanddvryv,1
22535538,zzmtsndchxbdmyxshamo,zzucufmtbxmihtplavko,1
22535539,zzmtsndchxbdmyxshamo,zzzfvqpmrvtwanddvryv,1


In [10]:
# Build a function that calculate the social distance between two participants

def dist_calculator(row):
    
    '''
    The input of the function should be one row of df_edge containing one pair of participant.
    The output of the function should be the social distance between two participants.
    '''
    
    dist = 0
    p1 = df_participant[df_participant['token'] == row[0]].iloc[0,:]
    p2 = df_participant[df_participant['token'] == row[1]].iloc[0,:]
    
    # Check if same age
    if abs(p1[2]-p2[2]) > 7:
        dist += 1
    
    # Check if same gender
    if p1[3] != p2[3]:
        dist += 1
    
    # Check if same marriage
    if p1[4] != p2[4]:
        dist += 1
        
    # Check if same marriage
    if p1[6] != p2[6]:
        dist += 1
    
    return dist

In [11]:
# Apply the distance calculator to calculate social distance between all participant pairs

if 1 == 0:
    dist_participants = df_edges.apply(dist_calculator, axis=1)

In [12]:
if 1 == 0:
    df_edges.rename(columns={0: "p1", 1: "p2"}, inplace = True)
    df_edges['social_dist'] = dist_participants

Now we have a dataframe storing edge information.

In [13]:
# Write the dataframe to csv

if 1 == 0:
    outpath = "C:\\Users\\kongl\\Desktop\\NYU Shanghai\\Fall 2021\\Network Analytics\\Project\participants_edgelist.csv"
    df_edges.to_csv(outpath, index = False)

In [14]:
# Read the participant edgelist into dataframe

path_edgelist = "C:\\Users\\kongl\\Desktop\\NYU Shanghai\\Fall 2021\\Network Analytics\\Project\participants_edgelist.csv"
df_edges = pd.read_csv(path_edgelist)

In [15]:
df_edges

,p1,p2,social_dist
0,aabfstzqyjjpmtxbmvkg,aaevujyayfdsvfsdtdju,1
1,aabfstzqyjjpmtxbmvkg,aafsazabzmeyvljckcoi,1
2,aabfstzqyjjpmtxbmvkg,aaloodukoptypzscbnrn,4
3,aabfstzqyjjpmtxbmvkg,aappcdtbnehthfwdzjjh,3
4,aabfstzqyjjpmtxbmvkg,aaqyqficqrloahhcvyay,1
...,...,...,...
22535536,zzloxgvjuwcsnrcncyhe,zzucufmtbxmihtplavko,3
22535537,zzloxgvjuwcsnrcncyhe,zzzfvqpmrvtwanddvryv,2
22535538,zzmtsndchxbdmyxshamo,zzucufmtbxmihtplavko,3
22535539,zzmtsndchxbdmyxshamo,zzzfvqpmrvtwanddvryv,1


We'll use this edge list to create a network.

## Participant Network

In [16]:
pip install nxviz

Note: you may need to restart the kernel to use updated packages.


In [17]:
# Import APIs for graph display

import nxviz as nv
import matplotlib.pyplot as plt

C:\Users\kongl\anaconda3\lib\site-packages\nxviz\__init__.py:18: UserWarning: 
nxviz has a new API! Version 0.7.2 onwards, the old class-based API is being
deprecated in favour of a new API focused on advancing a grammar of network
graphics. If your plotting code depends on the old API, please consider
pinning nxviz at version 0.7.2, as the new API will break your old code.

To check out the new API, please head over to the docs at
https://ericmjl.github.io/nxviz/ to learn more. We hope you enjoy using it!

(This deprecation message will go away in version 1.0.)

  warnings.warn(


In [18]:
# Build a NetworkX graph with from the DataFrame df_edges

G_tokens = nx.from_pandas_edgelist(df_edges, "p1", "p2", edge_attr="social_dist")

In [19]:
G_tokens.nodes()

NodeView(('aabfstzqyjjpmtxbmvkg', 'aaevujyayfdsvfsdtdju', 'aafsazabzmeyvljckcoi', 'aaloodukoptypzscbnrn', 'aappcdtbnehthfwdzjjh', 'aaqyqficqrloahhcvyay', 'aatlufbdhmigyxopxkhd', 'aauqlqeoaxoiddssinrq', 'abbcjjjqojwzzkwakszq', 'abbkfbizomqvaylxttti', 'abboxyvqhbswonhncfcw', 'abckotvzlhimxjsxpiiu', 'abcrfklxikuzejfnuxim', 'abelnhythhfaibmittdx', 'abrmlnsbmloeendqtguo', 'abtszytlskzoxtfzugdc', 'abtzpmdedwltnthozmxf', 'abupfcpyhgeyifvhnuot', 'abysvowcgygqsbwghheo', 'abzkhrpfilygyaqscrxt', 'acewiruqicqspzansinz', 'acgjrwfncwtzrmvfpuwg', 'acgpblpwbrzsdgmgrhgk', 'acijnftrjrycviejrumm', 'acpxujkjtzizlmrvgcjb', 'acsrrvomanbkdmaseuru', 'acstoufrytqpbvkwseje', 'acsxpdwmogdyskfnhvdl', 'acuebfdudpvrfklkfhzz', 'adahdpxttoswcwbunaiy', 'addvlfqfdifvdzorqnuv', 'adfgarxdtjatuzqwtlcn', 'adfpycckzcmkeyfvjyxo', 'adkeewpqqjqlhsbbzddl', 'adnniilwcnmtjibqpiqb', 'adspkiuzjdscurduulyt', 'advgkssuduzmoqipqlur', 'advqtfhinrpflznqidwz', 'adxmpiqedcfmhalncqzw', 'adypiaitvrutrkwolcxq', 'aecdbdbvemykvcjmstwr', 'aedex

The network turned out to be too large to display.<br>
But social distances between all participants are still very valuable information. <br>
We choose to shift to robot profile and come back later to see how we can integrate participant social distance information in our analysis.

## Robot Profile Clustering and Networking

In [20]:
# Recode variable robot_age

df['robot_age_recoded'] = 0
df.loc[(df['robot_age'] <= 32),'robot_age_recoded'] = "18-32"
df.loc[(df['robot_age'] > 32) & (df['robot_age'] <= 46),'robot_age_recoded'] = "33-46"
df.loc[(df['robot_age'] > 46) & (df['robot_age'] <= 60),'robot_age_recoded'] = "46-60"
df.loc[(df['robot_age'] > 60),'robot_age_recoded'] = ">60"

In [21]:
# Recode variable robot_review

df['robot_reviews_recoded'] = df['robot_reviews']
df.loc[(df['robot_reviews'] == 2) | (df['robot_reviews'] == 3),'robot_reviews_recoded'] = "2-3"
df.loc[(df['robot_reviews'] > 3),'robot_reviews_recoded'] = "many"

In [22]:
df['robot_id'] = df.index

In [23]:
profile_grp = df.groupby(['robot_age_recoded', 'robot_marital_status', 'robot_region', 'robot_gender', 'robot_rating', 'robot_reviews_recoded'])
df_robot = profile_grp.size().reset_index(name='count')

In [24]:
df_robot

,robot_age_recoded,robot_marital_status,robot_region,robot_gender,robot_rating,robot_reviews_recoded,count
0,18-32,married,MidWest,female,4,1,73
1,18-32,married,MidWest,female,4,2-3,61
2,18-32,married,MidWest,female,4,many,111
3,18-32,married,MidWest,female,5,1,93
4,18-32,married,MidWest,female,5,2-3,80
...,...,...,...,...,...,...,...
555,>60,single,West,male,4,many,19
556,>60,single,West,male,5,1,17
557,>60,single,West,male,5,2-3,16
558,>60,single,West,male,5,many,16


There are a total of 560 possible robot profiles.<br>
For each one profile, we can build a directional graph to find out how different participants react.

In [25]:
attributes_robot = df_robot.columns[:-1]

In [26]:
df_robot.loc[0,attributes_robot]

robot_age_recoded          18-32
robot_marital_status     married
robot_region             MidWest
robot_gender              female
robot_rating                   4
robot_reviews_recoded          1
Name: 0, dtype: object

In [27]:
# Build a function that label each robot

def label_robot(robot_id):
    
    '''
    The input of the function is the index of one robot profile in df_robot.
    The function would label all robot profiles in the original dataframe with the its index in df_robot.
    '''
    
    age = df_robot.loc[robot_id, attributes_robot[0]]
    marriage = df_robot.loc[robot_id, attributes_robot[1]]
    region = df_robot.loc[robot_id, attributes_robot[2]]
    gender = df_robot.loc[robot_id, attributes_robot[3]]
    rating = df_robot.loc[robot_id, attributes_robot[4]]
    review = df_robot.loc[robot_id, attributes_robot[5]]
    
    index = (df[attributes_robot[0]] == age) & (df[attributes_robot[1]] == marriage) & (df[attributes_robot[2]] == region) & (df[attributes_robot[3]] == gender) & (df[attributes_robot[4]] == rating) & (df[attributes_robot[5]] == review)
    df.loc[index,'robot_id'] = robot_id

In [28]:
# Label all robot profiles in the df

for robot_id in df_robot.index:
    label_robot(robot_id)

df['robot_id'].value_counts() # confirm that profiles has been correctly labeled

80     168
120    167
115    164
121    162
117    159
      ... 
559     10
546      9
552      9
554      8
553      3
Name: robot_id, Length: 560, dtype: int64

In [29]:
df

,token,age,gender,marital_status,state,region,homophily_random_1,homophily_random_2,world_random_1,answer.Risk,...,robot_region,robot_gender,robot_rating,robot_reviews,robot_distance,investment,world,robot_age_recoded,robot_reviews_recoded,robot_id
0,aabfstzqyjjpmtxbmvkg,64,female,married,Colorado,West,location,gender,rating,10001.0,...,West,female,5,2,0,20,1,>60,2-3,480
1,aabfstzqyjjpmtxbmvkg,64,female,married,Colorado,West,location,gender,rating,10001.0,...,NorthEast,female,5,2,1,20,1,>60,2-3,438
2,aabfstzqyjjpmtxbmvkg,64,female,married,Colorado,West,location,gender,rating,10001.0,...,South,male,5,2,2,20,1,>60,2-3,473
3,aabfstzqyjjpmtxbmvkg,64,female,married,Colorado,West,location,gender,rating,10001.0,...,MidWest,male,5,2,4,20,1,33-46,2-3,221
4,aabfstzqyjjpmtxbmvkg,64,female,married,Colorado,West,location,gender,rating,10001.0,...,South,male,4,2,5,20,1,18-32,2-3,120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33565,zzzfvqpmrvtwanddvryv,55,female,married,North Carolina,South,age,marital_status,rating,10001.0,...,South,female,5,38,0,30,1,46-60,many,327
33566,zzzfvqpmrvtwanddvryv,55,female,married,North Carolina,South,age,marital_status,rating,10001.0,...,South,female,5,34,1,25,1,18-32,many,47
33567,zzzfvqpmrvtwanddvryv,55,female,married,North Carolina,South,age,marital_status,rating,10001.0,...,South,female,5,25,2,15,1,18-32,many,117
33568,zzzfvqpmrvtwanddvryv,55,female,married,North Carolina,South,age,marital_status,rating,10001.0,...,West,male,5,46,4,20,1,18-32,many,138


Next we group by robot_id.

In [30]:
robot_id = df_robot.index
df_grp_rbt = df.groupby('robot_id')

In [31]:
# Build a function that form an edgelist for each robot and connected participants

def get_rbt_edgelist(rbt_id):
    
    '''
    The input of the function is a robot id.
    The output of the function is an edgelist in the form of a dataframe.
    '''
    
    df_rbt = df_grp_rbt.get_group(rbt_id)
    df_edge = pd.DataFrame()
    df_edge['token'] = list(df_rbt['token'])
    df_edge['robot_id'] = rbt_id
    df_edge['investment'] = list(df_rbt['investment'])
    df_edge['dist'] = list(df_rbt['robot_distance'])
    
    return df_edge

In [32]:
# Use the function to get all edgelists for all robots

edgelist_rbt = [get_rbt_edgelist(robot) for robot in robot_id]

Each element in the *edgelist_rbt* is a dataframe containing edge information to build a network.<br>
With edgelists, we can construct a network graph for each robot profile.

In [33]:
# Build a function that build a network graph for each robot profile with all the information we needed.

def rbt_network(rbt_id):
    
    '''G
    The input of the function should be a robot id.
    The output of the function should be a network graph with all valuable information in it.
    '''
    
    # build the framework of the graph
    df_rbt = df_grp_rbt.get_group(rbt_id)
    edgelist = edgelist_rbt[rbt_id]
    G_rbt = nx.from_pandas_edgelist(edgelist, "token", "robot_id", edge_attr="dist")
    
    # make the graph a bipartite one
    G_rbt.add_nodes_from(edgelist_rbt[rbt_id].token, bipartite = "token")
    G_rbt.add_nodes_from([rbt_id], bipartite = "robot")
    
    # add labels to each token node
    label_list = ['age', 'gender', 'marital_status', 'state', 'region', 'answer.Risk', 'robot_distance', 'world', 'investment']
    for label in label_list:
        nx.set_node_attributes(G_rbt, pd.Series(list(df_rbt[label]), index=list(df_rbt.token)).to_dict(), label)
    
    # add labels to robot node
    node_info = df_robot.loc[rbt_id]
    node_labels = node_info.index
    for node_label in node_labels:
        G_rbt.nodes[rbt_id][node_label] = node_info[node_label]
    
    return G_rbt

In [34]:
networks_rbt = [rbt_network(rbt_id) for rbt_id in df_robot.index]

Each element of the *networks_rbt* is a networkx graph containing all the information we need about participants and robot.<br>
With network graphs, we can analyze each robot profile. 

In [35]:
from copy import deepcopy

In [36]:
# Build a sample dictionary for later deepcopy

dict_sample = {}
colnames = ['num_token', 'avg_invest', 'avg_age', 'male', 'female', 'married', 'single', 'West', 'MidWest', 'South', 'Pacific', 'NorthEast', 'risk']
for col in colnames:
    dict_sample[col] = 0

In [37]:
# Build a function that analyze each robot profile given network graph

def rbt_analysis(rbt_id):
    
    '''
    The input of the function is robot id.
    The output of the function would be a dataframe containing the most important information that we are interested in.
    '''
    
    G_rbt = networks_rbt[rbt_id]
    
    # categorize participants(tokens) based on their social distance from the robot profile.
    tokens_d0 = [n for n, d in G_rbt.nodes(data = True) if d['bipartite'] == 'token' if d['robot_distance'] == 0]
    tokens_d1 = [n for n, d in G_rbt.nodes(data = True) if d['bipartite'] == 'token' if d['robot_distance'] == 1]
    tokens_d2 = [n for n, d in G_rbt.nodes(data = True) if d['bipartite'] == 'token' if d['robot_distance'] == 2]
    tokens_d4 = [n for n, d in G_rbt.nodes(data = True) if d['bipartite'] == 'token' if d['robot_distance'] == 4]
    tokens_d51 = [n for n, d in G_rbt.nodes(data = True) if d['bipartite'] == 'token' if (d['robot_distance'] == 5) & (d['world'] == 1)]
    tokens_d52= [n for n, d in G_rbt.nodes(data = True) if d['bipartite'] == 'token' if (d['robot_distance'] == 5) & (d['world'] == 2)]
    tokens_list = [tokens_d0, tokens_d1, tokens_d2, tokens_d4, tokens_d51, tokens_d52]
    
    # build a dataframe that summarizes all information we want
    df_sum = pd.DataFrame()
    df_sum['token_type'] = ['tokens_d0', 'tokens_d1', 'tokens_d2', 'tokens_d4', 'tokens_d51', 'tokens_d52']
    
    for i, tokens in enumerate(tokens_list):
        dict_sum = attr_sum(tokens, G_rbt)
        col_names = list(dict_sum.keys())
        col_values = list(dict_sum.values())
        df_sum.loc[i, col_names] = col_values
        
    return df_sum

In [38]:
# Build a function to summarize participant attributes for each type of token in each network we got

def attr_sum(tokens, G_rbt):
    
    '''
    This function will be used in the following rbt_analysis function.
    The input of the function is a list of token names for one specific type of token, e.g., tokens that are 1 distance away from the robot profile.
    The output of the function is a dictionary containing the information we want.
    '''
    
    # Build a dictionary for data storage
    dict_sum = deepcopy(dict_sample)
    
    # Summarize and record attribute information of the tokens
    for n in tokens:
        for key, value in G_rbt.nodes[n].items():
            if key == 'investment':
                dict_sum['avg_invest'] += value
            if key == 'age':
                dict_sum['avg_age'] += value
            if key == 'gender':
                dict_sum[value] += 1
            if key == 'marital_status':
                dict_sum[value] += 1
            if key == 'region':
                dict_sum[value] += 1
            if key == 'answer.Risk':
                dict_sum['risk'] += value
    
    # Calculate average age and average risk
    num = len(tokens)
    if num != 0:
        dict_sum['num_token'] = num
        dict_sum['avg_invest'] /= num
        dict_sum['avg_age'] /= num
        dict_sum['risk'] /= num
    
    return dict_sum

In [39]:
analyses_rbt = [rbt_analysis(rbt_id) for rbt_id in df_robot.index]

Each element of *analyses_rbt* is a dataframe summarizing information about participants who invested one unique robot profile in the game.

In [40]:
analyses_rbt[0]

,token_type,num_token,avg_invest,avg_age,male,female,married,single,West,MidWest,South,Pacific,NorthEast,risk
0,tokens_d0,8.0,24.625000,27.375000,0.0,8.0,8.0,0.0,0.0,8.0,0.0,0.0,0.0,5212.750000
1,tokens_d1,15.0,12.533333,34.333333,0.0,15.0,8.0,7.0,0.0,11.0,2.0,0.0,2.0,5920.333333
2,tokens_d2,23.0,11.521739,36.652174,7.0,16.0,9.0,14.0,1.0,8.0,3.0,6.0,5.0,4875.173913
3,tokens_d4,16.0,13.437500,46.750000,16.0,0.0,0.0,16.0,0.0,0.0,5.0,6.0,5.0,4036.187500
4,tokens_d51,7.0,8.285714,45.142857,7.0,0.0,0.0,7.0,1.0,0.0,3.0,3.0,0.0,6071.857143
5,tokens_d52,3.0,10.000000,39.000000,3.0,0.0,0.0,3.0,1.0,0.0,1.0,0.0,1.0,10001.000000


## Robot Profile Analysis

We get a list of edgelist for all 560 robot profiles and have built a small network graphs for all them. <br>
We also separately analyzed investors of each robot profile. <br>
However, how we make comparison across robots remains a question.

In [41]:
df_robot.head(5)

,robot_age_recoded,robot_marital_status,robot_region,robot_gender,robot_rating,robot_reviews_recoded,count
0,18-32,married,MidWest,female,4,1,73
1,18-32,married,MidWest,female,4,2-3,61
2,18-32,married,MidWest,female,4,many,111
3,18-32,married,MidWest,female,5,1,93
4,18-32,married,MidWest,female,5,2-3,80


We can see that some robots only differ in one or two of the six attributes while some robots differ in all.<br>
The most reasonable way to compare robot nodes is to compare those that only differ in one variable. <br>
So, if two robot profiles differ in only one variable, we consider them as "comparable", therefore forming a connection between them.<br>
In this way, we can get a network with 560 robot profiles as nodes and the "comparability" of profile as edges.<br>
For each node, all nodes that are 1 distance aways from it have 1 different variable and all nodes that are 2 distance away from it have two different variables.

To achieve this, we need to build an edgelist with two columns representing nodes at two side of each edge and one column representing the only one different variable between the two edges.

In [42]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [43]:
# Get all possible robot pairs 

paired_robots = list(itertools.combinations(list(df_robot.index),2))
df_rbt_pairs = pd.DataFrame(paired_robots)

In [44]:
# Restructure the dataframe for edge storage

df_rbt_edges = pd.DataFrame(df_rbt_pairs.groupby([0,1]).size())
df_rbt_edges.rename(columns={0:'num_diff'}, inplace=True)
df_rbt_edges.reset_index(inplace=True)
df_rbt_edges

,0,1,num_diff
0,0,1,1
1,0,2,1
2,0,3,1
3,0,4,1
4,0,5,1
...,...,...,...
156515,556,558,1
156516,556,559,1
156517,557,558,1
156518,557,559,1


In [45]:
robot_attrs = ['robot_age_recoded', 'robot_marital_status', 'robot_region', 'robot_gender', 'robot_rating', 'robot_reviews_recoded']

In [46]:
# Build a function that detects number of attribute differences between two robots

def diff_detector(row):
    
    '''
    The input of the function should be one row of df_rbt_edge containing one pair of robots.
    The output of the function should be the number of different attributes between two robots.   
    '''
    
    id1 = row[0]
    id2 = row[1]
    rbt1 = df_robot.loc[id1,robot_attrs]
    rbt2 = df_robot.loc[id2,robot_attrs]
    comparison = (rbt1 == rbt2).value_counts()
    
    if False in comparison.index:
        num_diff = comparison[False]
    else:
        num_diff = 0
    
    return num_diff

In [47]:
# Apply the difference detector to find number of difference attributes between all robot pairs

if 1 == 0:
    diff_rbt = df_rbt_edges.apply(diff_detector, axis=1)

In [48]:
# Remove all pairs with values not equal to 1

if 1 == 0:
    df_rbt_edges['num_diff'] = diff_rbt
    df_rbt_diff = df_rbt_edges[df_rbt_edges['num_diff'] == 1]
    df_rbt_diff.reset_index(inplace = True, drop = True)

In [49]:
# Build a function that find which variable is different between the two robots

def find_rbt_diff(row):
    
    '''
    The input of the function should be one row of df_rbt_diff containing one pair of robots.
    The output of the function should the different attributes between two robots.   
    '''
    
    id1 = row[0]
    id2 = row[1]
    rbt1 = df_robot.loc[id1,robot_attrs]
    rbt2 = df_robot.loc[id2,robot_attrs]
    comparison = (rbt1 == rbt2)
    
    diff = comparison[comparison == False].index[0]
    
    return diff

In [50]:
# Apply the difference finder to find the different attribute for each robot pairs

if 1 == 0:
    diff_rbt = df_rbt_diff.apply(find_rbt_diff, axis=1)

In [51]:
# Modify the structure of the dataframe

if 1 == 0:
    df_rbt_diff['difference'] = diff_rbt
    df_rbt_diff.drop(columns = 'num_diff', inplace = True)
    df_rbt_diff.rename(columns={0: "rbt1", 1: "rbt2"}, inplace = True)

In [52]:
# Write robot edgelist to csv file

if 1 == 0:
    outpath = "C:\\Users\\kongl\\Desktop\\NYU Shanghai\\Fall 2021\\Network Analytics\\Project\\robots_edgelist.csv"
    df_rbt_diff.to_csv(outpath, index = False)

In [53]:
# Read the participant edgelist into dataframe

path_edgelist = "C:\\Users\\kongl\\Desktop\\NYU Shanghai\\Fall 2021\\Network Analytics\\Project\\robots_edgelist.csv"
df_rbt_edges = pd.read_csv(path_edgelist)

In [54]:
df_rbt_edges

,rbt1,rbt2,difference
0,0,1,robot_reviews_recoded
1,0,2,robot_reviews_recoded
2,0,3,robot_rating
3,0,7,robot_gender
4,0,14,robot_region
...,...,...,...
3235,554,557,robot_rating
3236,555,558,robot_rating
3237,556,557,robot_reviews_recoded
3238,556,558,robot_reviews_recoded


In [55]:
df_rbt_edges['difference'].value_counts()

robot_region             1120
robot_age_recoded         840
robot_reviews_recoded     480
robot_marital_status      280
robot_gender              280
robot_rating              240
Name: difference, dtype: int64

Now we can build a graph based on this edgelist.

In [56]:
# Build a NetworkX graph with from the DataFrame df_rbt_edges

G_robots = nx.from_pandas_edgelist(df_rbt_edges, "rbt1", "rbt2", edge_attr="difference")

Next, we would like to compare investment differences for each type of token for each pair of nodes.
The way we make the comparison is to calculate the confidence interval of mean difference.

In [57]:
import statsmodels.stats.api as sms

In [58]:
# Build a sample dictionary for later deepcopy

dict_sample = {}
colnames = ['rbt1_level', 'rbt2_level', 'token_d0', 'token_d1', 'token_d2', 'token_d4', 'token_d51', 'token_d52']
for col in colnames:
    dict_sample[col] = 0

In [59]:
# Build a function that calculates the mean differences for all token types in all robot pairs

def ci_calculator(rbt1, rbt2, difference):
    
    '''
    The input of the function are ids of robot1, robot2, and the different variable between them.
    The function will be a dictionary of 
    '''
    
    dict_rbt_edge = deepcopy(dict_sample)
    dict_rbt_edge['rbt1_level'] = df_robot.loc[rbt1][difference]
    dict_rbt_edge['rbt2_level'] = df_robot.loc[rbt2][difference]
    
    df_rbt1 = df_grp_rbt.get_group(rbt1)
    df_rbt2 = df_grp_rbt.get_group(rbt2)
    distances = [0, 1, 2, 4, 5]
    CI_list = []
    
    for dist in distances:
        if dist != 5:
            invest1 = df_rbt1[df_rbt1['robot_distance'] == dist]['investment']
            invest2 = df_rbt2[df_rbt2['robot_distance'] == dist]['investment']
            if (len(invest1) != 0) & (len(invest2) != 0):
                cm = sms.CompareMeans(sms.DescrStatsW(invest1), sms.DescrStatsW(invest2))
                CI_list.append(list(cm.tconfint_diff(usevar='unequal')))
            else:
                CI_list.append(np.NaN)
        else:
            df_rbt1_d5 = df_rbt1[df_rbt1['robot_distance'] == dist]
            df_rbt2_d5 = df_rbt2[df_rbt2['robot_distance'] == dist]
            invest1_w1 = df_rbt1_d5[df_rbt1_d5['world'] == 1]['investment']
            invest2_w1 = df_rbt1_d5[df_rbt1_d5['world'] == 1]['investment']
            if (len(invest1_w1) != 0) & (len(invest2_w1) != 0):
                cm = sms.CompareMeans(sms.DescrStatsW(invest1_w1), sms.DescrStatsW(invest2_w1))
                CI_list.append(list(cm.tconfint_diff(usevar='unequal')))
            else:
                CI_list.append(np.NaN)
                
            invest1_w2 = df_rbt1_d5[df_rbt1_d5['world'] == 2]['investment']
            invest2_w2 = df_rbt1_d5[df_rbt1_d5['world'] == 2]['investment']
            if (len(invest1_w2) != 0) & (len(invest2_w2) != 0):
                cm = sms.CompareMeans(sms.DescrStatsW(invest1_w2), sms.DescrStatsW(invest2_w2))
                CI_list.append(list(cm.tconfint_diff(usevar='unequal')))
            else:
                CI_list.append(np.NaN)
    
    dict_rbt_edge['token_d0'] = CI_list[0]
    dict_rbt_edge['token_d1'] = CI_list[1]
    dict_rbt_edge['token_d2'] = CI_list[2]
    dict_rbt_edge['token_d4'] = CI_list[3]
    dict_rbt_edge['token_d51'] = CI_list[4]
    dict_rbt_edge['token_d52'] = CI_list[5]
    
    return dict_rbt_edge

In [60]:
# Apply the function and add the output as new labels to edges between robots

for u, v, d in G_robots.edges(data = True):
    dict_diff = ci_calculator(u, v, d["difference"])
    for key, value in dict_diff.items():
        G_robots.edges[u,v][key] = value

C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1006: RuntimeWarning: invalid value encountered in double_scalars
  sem1 = d1._var / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1007: RuntimeWarning: invalid value encountered in double_scalars
  sem2 = d2._var / (d2.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1006: RuntimeWarning: invalid value encountered in double_scalars
  sem1 = d1._var / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\st

C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1009: RuntimeWarning: invalid value encountered in double_scalars
  z1 = (sem1 / semsum) ** 2 / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1010: RuntimeWarning: invalid value encountered in double_scalars
  z2 = (sem2 / semsum) ** 2 / (d2.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1009: RuntimeWarning: invalid value encountered in double_scalars
  z1 = (sem1 / semsum) ** 2 / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1010: RuntimeWarning: invalid value encountered in double_scalars
  z2 = (sem2 / semsum) ** 2 / (d2.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\stat

C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1006: RuntimeWarning: invalid value encountered in double_scalars
  sem1 = d1._var / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1007: RuntimeWarning: invalid value encountered in double_scalars
  sem2 = d2._var / (d2.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1006: RuntimeWarning: invalid value encountered in double_scalars
  sem1 = d1._var / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\st

C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1007: RuntimeWarning: invalid value encountered in double_scalars
  sem2 = d2._var / (d2.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1006: RuntimeWarning: invalid value encountered in double_scalars
  sem1 = d1._var / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1007: RuntimeWarning: invalid value encountered in double_scalars
  sem2 = d2._var / (d2.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\st

C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1007: RuntimeWarning: invalid value encountered in double_scalars
  sem2 = d2._var / (d2.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1006: RuntimeWarning: invalid value encountered in double_scalars
  sem1 = d1._var / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1007: RuntimeWarning: invalid value encountered in double_scalars
  sem2 = d2._var / (d2.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\st

...

C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1006: RuntimeWarning: invalid value encountered in double_scalars
  sem1 = d1._var / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1007: RuntimeWarning: invalid value encountered in double_scalars
  sem2 = d2._var / (d2.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1006: RuntimeWarning: invalid value encountered in double_scalars
  sem1 = d1._var / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\st

C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1006: RuntimeWarning: invalid value encountered in double_scalars
  sem1 = d1._var / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1007: RuntimeWarning: invalid value encountered in double_scalars
  sem2 = d2._var / (d2.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1006: RuntimeWarning: invalid value encountered in double_scalars
  sem1 = d1._var / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\st

C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1006: RuntimeWarning: invalid value encountered in double_scalars
  sem1 = d1._var / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1007: RuntimeWarning: invalid value encountered in double_scalars
  sem2 = d2._var / (d2.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1006: RuntimeWarning: invalid value encountered in double_scalars
  sem1 = d1._var / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1007: RuntimeWarning: invalid value encountered in double_scalars
  sem2 = d2._var / (d2.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWar

C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1007: RuntimeWarning: invalid value encountered in double_scalars
  sem2 = d2._var / (d2.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1007: RuntimeWarning: invalid value encountered in double_scalars
  sem2 = d2._var / (d2.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda

C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1006: RuntimeWarning: invalid value encountered in double_scalars
  sem1 = d1._var / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:1006: RuntimeWarning: invalid value encountered in double_scalars
  sem1 = d1._var / (d1.nobs - 1)
C:\Users\kongl\anaconda3\lib\site-packages\statsmodels\stats\weightstats.py:979: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(d1._var / (d1.nobs - 1) + d2._var / (d2.nobs - 1))
C:\Users\kongl\anaconda

We can see that all mean differences in investment were calculated and added to edges as labels.<br>
As you can see below, we can check how investments differ between participants who are 0 distance away when two robots differ in gender.

In [61]:
for u, v, d in G_robots.edges(data = True):
    if d['difference'] == 'robot_gender':
        print(d['token_d0'])

[-31.23978546051093, 60.48978546051093]
[-4.738775529730914, 18.460997751953137]
[-43.950651089027275, 13.950651089027279]
[-18.02326416584551, 19.845486388067734]
[-6.441733116372072, 24.95601883065779]
[-17.963495368135966, 1.3444477490883475]
[-40.50011036237939, 9.881062743331771]
[nan, nan]
[-4.54102404140344, 8.387690708070107]
[nan, nan]
[-5.209106156544174, 7.209106156544174]
[nan, nan]
[-10.197602927282, 17.283317212996288]
[-11.238208639320453, 13.808344385926786]
[-11.976418211744377, 11.434751545077713]
[-11.976693783529639, 6.022148328984185]
[-35.70371379106893, 41.70371379106893]
[-10.61688086426142, 4.528645570143775]
[-17.522648030143493, 10.322648030143489]
[6.7174879353973616, 28.28251206460264]
[nan, nan]
[-10.60033901651603, 11.686053302230311]
[-3.891120750808202, 16.5411207508082]
[-11.422491826850113, 17.59896241508541]
[-4.885820734035924, 19.962743810959005]
[-12.761341112649669, 7.961341112649672]
[-11.327332127851042, 6.716221016739931]
[-15.898698114736803,

## Predictive Model building

In this section, we want to build a model that, given the robot profile and distance between participant and profile, predict how much investment(trust) would a certain participant put on the profile.

In [62]:
import seaborn as sns
import statsmodels.api as sm
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

In [63]:
# Get the information we want to use for modeling

select_var = ['robot_age', 'robot_marital_status', 'robot_region', 'robot_gender', 'robot_reviews_recoded', 'robot_rating', 'robot_distance', 'world', 'investment']
df_model = df.loc[:,select_var]
df_model.loc[(df_model['robot_distance'] == 5) & (df_model['world'] == 2), 'robot_distance'] = 6
df_model.loc[df_model['robot_rating'] == 'not_available', 'robot_rating'] = 0
df_model.drop(columns = "world", inplace = True)

In [270]:
# Apply one-hot encoding to categorical variables and get predictor and outcome variables

df_encoded = pd.get_dummies(df_model, columns = ['robot_gender','robot_marital_status','robot_region','robot_rating','robot_reviews_recoded','robot_distance'], drop_first = True)
X = df_encoded.drop(columns = 'investment')
y = df_encoded.investment

In [329]:
# Use a 10-fold cross validation to evaluate linear regression model

cv = KFold(n_splits=10, random_state=10, shuffle=True)
model_reg = LinearRegression()
scores_reg = cross_val_score(model_reg, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
print(scores_reg)

[-9.14590485 -9.18686348 -9.22273823 -9.15897287 -9.12262144 -8.91898609
 -9.23061282 -9.33028502 -9.17819689 -9.20447614]


In [330]:
# Split predictor and outcome variables into training set and test set and fit the regression model

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 1216)
regression = sm.OLS(y_train, sm.add_constant(X_train)).fit()
print(regression.summary())

                            OLS Regression Results                            
Dep. Variable:             investment   R-squared:                       0.126
Model:                            OLS   Adj. R-squared:                  0.126
Method:                 Least Squares   F-statistic:                     212.5
Date:                Wed, 15 Dec 2021   Prob (F-statistic):               0.00
Time:                        13:38:23   Log-Likelihood:                -92781.
No. Observations:               23499   AIC:                         1.856e+05
Df Residuals:                   23482   BIC:                         1.857e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

In [331]:
# Predict with the fitted model and evaluate model performance

y_pred_lr = regression.predict(sm.add_constant(X_test))
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred_lr)))

12.492467538652658


The effect of each variable is consistent with discoveries and conjectures in the initial paper.<br>
But the predictive power of the multiple linear regression model is not very satisfying. <br>
We can try use other models to improve the performance.

In [68]:
from sklearn.metrics import *
from catboost import Pool, CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [69]:
# Fit the regression tree 

regression_tree = DecisionTreeRegressor(max_depth=6, random_state = 0)
regression_tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=6, random_state=0)

In [70]:
y_pred_rt = regression_tree.predict(X_test)
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred_rt)))

12.688663121579715


In [71]:
# Build a catboost regressor

model_cat = CatBoostRegressor(loss_function='RMSE')
hyperparameters = {'iterations':[50, 100, 200], 'max_depth': [4,6,8], 'learning_rate':[0.01, 0.05, 0.1]}
grid = GridSearchCV(estimator=model_cat, param_grid = hyperparameters, cv = 5, n_jobs=-1)
grid.fit(X_train, y_train)

0:	learn: 13.2861966	total: 56.1ms	remaining: 5.55s
1:	learn: 13.1737672	total: 59.2ms	remaining: 2.9s
2:	learn: 13.0848207	total: 62.1ms	remaining: 2.01s
3:	learn: 13.0039447	total: 64.9ms	remaining: 1.56s
4:	learn: 12.9381184	total: 67.7ms	remaining: 1.29s
5:	learn: 12.8838750	total: 71.3ms	remaining: 1.12s
6:	learn: 12.8389322	total: 74.5ms	remaining: 989ms
7:	learn: 12.7990497	total: 77.5ms	remaining: 891ms
8:	learn: 12.7630641	total: 80.2ms	remaining: 811ms
9:	learn: 12.7328600	total: 82.9ms	remaining: 746ms
10:	learn: 12.7092924	total: 85.8ms	remaining: 695ms
11:	learn: 12.6873431	total: 88.5ms	remaining: 649ms
12:	learn: 12.6647005	total: 91.3ms	remaining: 611ms
13:	learn: 12.6466071	total: 94.1ms	remaining: 578ms
14:	learn: 12.6318805	total: 97.1ms	remaining: 550ms
15:	learn: 12.6191193	total: 99.7ms	remaining: 523ms
16:	learn: 12.6101782	total: 102ms	remaining: 500ms
17:	learn: 12.6007469	total: 105ms	remaining: 478ms
18:	learn: 12.5919819	total: 107ms	remaining: 458ms
19:	lea

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostRegressor object at 0x000002710B034F40>,
             n_jobs=-1,
             param_grid={'iterations': [50, 100, 200],
                         'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [4, 6, 8]})

In [72]:
grid.best_params_

{'iterations': 100, 'learning_rate': 0.1, 'max_depth': 4}

In [73]:
model_cat = CatBoostRegressor(
    **grid.best_params_,
    verbose=False
)
model_cat.fit(
        X_train, y_train,
        logging_level='Verbose',  
        plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [74]:
y_pred_cat = model_cat.predict(X_test)
rmse = (np.sqrt(metrics.mean_squared_error(y_test, y_pred_cat)))
r2 = r2_score(y_test, y_pred_cat)
print('Testing performance')
print('RMSE: {:.2f}'.format(rmse))
print('R2: {:.2f}'.format(r2))

Testing performance
RMSE: 12.44
R2: 0.14


CatBoost Regression model didn't have a better performance.

## Environment Diversity information extraction

In our previous model, our predictor variables are focusing only on robot profile characteristics.<br>
What our previous model predicts is, given a robot profile, how much investment(trust) would a customer that is x distance away from the robot profile give to the robot.<br>
In this experiment and other real life cases, there are information other than robot profile itself that we are interested in.<br>
Participants or customers don't only get one profile a time and choose which amount to trust to put on it. They are given multiple profiles to choose and the trust participants put on one profile is definitely affected by other profiles that coexist.<br>
Therefore, we want to extract information that represents the diverisity of the environment when the profile is displayed to participants.

In [77]:
df_grp_token = df.groupby('token')

In [172]:
# Build a fucntion that calculate the diversity for each token-robot group

def find_diversity(token):
    
    '''
    The input of the function is the token name.
    The output of the function a list including measurements of diversity. 
    '''
    
    df_token = df_grp_token.get_group(token)
    
    # calulate age diversity
    variance_age = np.std(df_token['robot_age'])
    
    # calculate gender diversity
    gender_count = df_token['robot_gender'].value_counts()
    if "male" in gender_count.index:
        num_male = gender_count['male']
    else:
        num_male = 0
    percent_male = num_male/5
    
    # calculate marriage diversity
    marriage_count = df_token['robot_marital_status'].value_counts()
    if "married" in marriage_count.index:
        num_married = marriage_count['married']
    else:
        num_married = 0
    percent_married = num_married/5
    
    # calculate region diversity
    num_region = len(df_token['robot_region'].value_counts())
    
    diversity_measures = [token, variance_age, percent_male, percent_married, num_region]
    
    return diversity_measures

In [173]:
# Calculate the environment diversity for each participant

diversity_attrs = [find_diversity(token) for token in list_participants]
df_diverse = pd.DataFrame(diversity_attrs)
df_diverse.columns = ['token', 'robot_var_age', 'robot_percent_male', 'robot_percent_married', 'robot_unique_regions']

In [382]:
# Add the environment diversity variables by merging our previous dataframe and diversity dataframe

df_encoded['token'] = df['token']
df_added = df_encoded.merge(df_diverse, on='token', how='left')

**df_added** contains 6 robot profile attributes, 4 environment diversity attributes, and 1 social distance attributes.<br>
With 4 environment diversity attributes added, we can build a new linear regression model.

## Modeling with Diversity Variables Added

We not only added the diversity variables, but also two participant variables, *gender* and *marital status*.

In [384]:
df_added['robot_percent_male'] = df_added['robot_percent_male'] * 5
df_added['robot_percent_married'] = df_added['robot_percent_married'] * 5

In [386]:
# Add the interactive variables

df_added['gender*percent_gender'] = df_added['robot_gender_male'] * df_added['robot_percent_male']
df_added['married*percent_married'] = df_added['robot_marital_status_single'] * df_added['robot_percent_married']

In [387]:
df_added['subject_male'] = df['gender']
df_added['subject_single'] = df['marital_status']
df_added['subject_male'] = (df_added['subject_male'] == 'male')*1
df_added['subject_single'] = (df_added['subject_single'] == 'single')*1

In [389]:
df_added

,robot_age,investment,robot_gender_male,robot_marital_status_single,robot_region_NorthEast,robot_region_Pacific,robot_region_South,robot_region_West,robot_rating_4,robot_rating_5,...,robot_distance_6,token,robot_var_age,robot_percent_male,robot_percent_married,robot_unique_regions,gender*percent_gender,married*percent_married,subject_male,subject_single
0,68,20,0,0,0,0,0,1,0,1,...,0,aabfstzqyjjpmtxbmvkg,16.007498,3.0,3.0,4,0.0,0.0,0,0
1,64,20,0,0,1,0,0,0,0,1,...,0,aabfstzqyjjpmtxbmvkg,16.007498,3.0,3.0,4,0.0,0.0,0,0
2,67,20,1,0,0,0,1,0,0,1,...,0,aabfstzqyjjpmtxbmvkg,16.007498,3.0,3.0,4,3.0,0.0,0,0
3,37,20,1,1,0,0,0,0,0,1,...,0,aabfstzqyjjpmtxbmvkg,16.007498,3.0,3.0,4,3.0,3.0,0,0
4,31,20,1,1,0,0,1,0,1,0,...,0,aabfstzqyjjpmtxbmvkg,16.007498,3.0,3.0,4,3.0,3.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33565,56,30,0,0,0,0,1,0,0,1,...,0,zzzfvqpmrvtwanddvryv,12.986146,2.0,2.0,3,0.0,0.0,0,0
33566,28,25,0,0,0,0,1,0,0,1,...,0,zzzfvqpmrvtwanddvryv,12.986146,2.0,2.0,3,0.0,0.0,0,0
33567,22,15,0,1,0,0,1,0,0,1,...,0,zzzfvqpmrvtwanddvryv,12.986146,2.0,2.0,3,0.0,2.0,0,0
33568,22,20,1,1,0,0,0,1,0,1,...,0,zzzfvqpmrvtwanddvryv,12.986146,2.0,2.0,3,2.0,2.0,0,0


In [390]:
X_d = df_added.drop(columns = ['token', 'investment'])
y_d = df_added.investment

In [373]:
# Use a 10-fold cross validation to evaluate linear regression model

cv = KFold(n_splits=10, random_state=10, shuffle=True)
model_reg_d = LinearRegression()
scores_reg_d = cross_val_score(model_reg_d, X_d, y_d, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
print(scores_reg_d)

[-9.13624162 -9.20392851 -9.2015395  -9.14667379 -9.09646898 -8.89762414
 -9.20877442 -9.32153871 -9.15799122 -9.1780281 ]


In [391]:
# Split predictor and outcome variables into training set and test set and fit the regression model

X_d_train, X_d_test, y_d_train, y_d_test = train_test_split(X_d, y_d, test_size=0.3, random_state = 1216)
regression_d = sm.OLS(y_d_train, sm.add_constant(X_d_train)).fit()
print(regression_d.summary())

                            OLS Regression Results                            
Dep. Variable:             investment   R-squared:                       0.130
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     146.3
Date:                Wed, 15 Dec 2021   Prob (F-statistic):               0.00
Time:                        17:10:26   Log-Likelihood:                -92732.
No. Observations:               23499   AIC:                         1.855e+05
Df Residuals:                   23474   BIC:                         1.857e+05
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             

In [336]:
# Predict with the fitted model and evaluate model performance

y_d_pred_lr = regression_d.predict(sm.add_constant(X_d_test))
print(np.sqrt(metrics.mean_squared_error(y_d_test, y_d_pred_lr)))

12.496107273377117


We can compare the summaries of our original model and the model with diversity included.

In [287]:
print(regression.summary())

                            OLS Regression Results                            
Dep. Variable:             investment   R-squared:                       0.126
Model:                            OLS   Adj. R-squared:                  0.126
Method:                 Least Squares   F-statistic:                     212.5
Date:                Wed, 15 Dec 2021   Prob (F-statistic):               0.00
Time:                        13:31:56   Log-Likelihood:                -92781.
No. Observations:               23499   AIC:                         1.856e+05
Df Residuals:                   23482   BIC:                         1.857e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             